#Codigo Bruto


In [ ]:
import os
import zipfile
import urllib.request
import random
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout

# URL do dataset e configuração de pastas
DATA_URL = (
    "https://download.microsoft.com/download/3/E/1/"
    "3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip"
)
EXTRACT_PATH = "cats_and_dogs"

# Função para baixar e extrair os dados
def download_and_extract_data(url, extract_path):
    zip_path = "dataset.zip"
    if not os.path.exists(extract_path):
        print("Baixando dataset...")
        urllib.request.urlretrieve(url, zip_path)
        print("Extraindo dataset...")
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(extract_path)
        os.remove(zip_path)
        print("Dataset preparado!")
    else:
        print("Dataset já está preparado.")

download_and_extract_data(DATA_URL, EXTRACT_PATH)

# Configuração do gerador de dados
base_dir = os.path.join(EXTRACT_PATH, "PetImages")
datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    validation_split=0.2  # 20% dos dados para validação
)

# Criar geradores de treino e validação
train_generator = datagen.flow_from_directory(
    base_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='training'
)

validation_generator = datagen.flow_from_directory(
    base_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)

# Carregar o modelo VGG16 pré-treinado
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Congelar as camadas da base do modelo

# Construir o modelo com transferência de aprendizado
model = Sequential([
    base_model,
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Camada de saída para classificação binária
])

# Compilar o modelo
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

# Treinamento do modelo
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=10,
    verbose=1
)

# Função para plotar os resultados
def plot_training_history(history):
    plt.figure(figsize=(12, 4))
    # Loss
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Loss - Treino')
    plt.plot(history.history['val_loss'], label='Loss - Validação')
    plt.legend()
    plt.title('Loss durante o Treinamento')
    # Accuracy
    plt.subplot(1, 2, 2)
    plt.plot(history.history['accuracy'], label='Accuracy - Treino')
    plt.plot(history.history['val_accuracy'], label='Accuracy - Validação')
    plt.legend()
    plt.title('Accuracy durante o Treinamento')
    plt.show()

# Plotar histórico do treinamento
plot_training_history(history)

# Avaliação final no conjunto de validação
loss, accuracy = model.evaluate(validation_generator, verbose=0)
print(f"Loss final: {loss:.4f}")
print(f"Accuracy final: {accuracy:.4f}")


Dataset já está preparado.
Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ vgg16 (Functional)                   │ (None, 7, 7, 512)           │      14,714,688 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 25088)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 256)                 │       6,422,784 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │             257 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 21,137,729 (80.63 MB)

 Trainable params: 6,423,041 (24.50 MB)

 Non-trainable params: 14,714,688 (56.13 MB)

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


 23/625 ━━━━━━━━━━━━━━━━━━━━ 3:24:16 20s/step - accuracy: 0.5501 - loss: 2.1788

In [ ]:
import os
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
from google.colab import files

def predict_image(image_path):
    try:
        img = image.load_img(image_path, target_size=(224, 224))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        preds = model.predict(x)
        if preds[0][0] >= 0.5:
            return "Cachorro"
        else:
            return "Gato"
    except Exception as e:
        return f"Erro na predição: {e}"

# Fazer upload do arquivo
uploaded = files.upload()

# Verificar se o arquivo foi carregado
if uploaded:
    uploaded_file_path = list(uploaded.keys())[0]
    prediction = predict_image(uploaded_file_path)
    print(f"Predição: {prediction}")
else:
    print("Nenhum arquivo foi carregado.")


Saving OIP.jpeg to OIP.jpeg
Predição: Erro na predição: name 'model' is not defined
